# 回転の推定II:異方性誤差

In [1]:
from pathlib import Path
import sys
from itertools import product

import numpy as np
from scipy import linalg
from scipy.stats import random_correlation, special_ortho_group
from scipy.spatial.transform import Rotation

sys.path.append('../libs')
import util

In [2]:
A_bar = util.load_point_cloud(Path('../bunny/data/bun180.ply').resolve())
points_num = A_bar.shape[1]
print(points_num)

40251


In [ ]:
util.plot_3d(A_bar)

In [ ]:
sigma = np.array([1., 1., 2.])
cov = np.outer(sigma, sigma) * random_correlation.rvs((1.2, 0.8, 1.0))
util.plot_3d(np.random.multivariate_normal(np.zeros(3), cov, 10000).T)

In [3]:
sigma = np.array([1., 1., 3.])
cov_a = np.outer(sigma, sigma) * random_correlation.rvs((0.5, 1.2, 1.3))
noise_level = 3e-3
A = A_bar + noise_level * np.random.multivariate_normal(np.zeros(3), cov_a, points_num).T

In [ ]:
util.plot_3d(A)

In [4]:
ideal_R = special_ortho_group.rvs(3)
print(ideal_R)

[[ 0.8056051  -0.24243616  0.54057851]
 [ 0.44925333 -0.34485353 -0.82416472]
 [ 0.38622774  0.906808   -0.16890051]]


In [5]:
sigma = np.array([2., 1., 1.])
cov_a_prime = np.outer(sigma, sigma) * random_correlation.rvs((0.1, 0.2, 2.7))
A_prime = ideal_R @ A_bar + noise_level * np.random.multivariate_normal(np.zeros(3), cov_a_prime, points_num).T

In [ ]:
util.plot_3d(A_prime)

## 特異値分解の場合

In [6]:
R1 = util.estimate_R_using_SVD(A, A_prime)
print('error:', util.eval_R_error(R1, ideal_R))

error: 0.0011221782784201111


In [ ]:
util.plot_3d_multi(R1 @ A, A_prime)

## 5.3 四元数表現による回転推定

In [7]:
Xi = np.stack([
    np.stack([
        A_prime[0] - A[0],
        np.zeros(points_num),
        -(A_prime[2] + A[2]),
        A_prime[1] + A[1]
    ]),
    np.stack([
        A_prime[1] - A[1],
        A_prime[2] + A[2],
        np.zeros(points_num),
        -(A_prime[0] + A[0])
    ]),
    np.stack([
        A_prime[2] - A[2],
        -(A_prime[1] + A[1]),
        A_prime[0] + A[0],
        np.zeros(points_num)
    ])
])
print(Xi.shape)

(3, 4, 40251)


In [8]:
T = np.array([
    [
        [-1,  0,  0,  1,  0,  0],
        [ 0,  0,  0,  0,  0,  0],
        [ 0,  0, -1,  0,  0, -1],
        [ 0,  1,  0,  0,  1,  0]
    ], [
        [ 0, -1,  0,  0,  1,  0],
        [ 0,  0,  1,  0,  0,  1],
        [ 0,  0,  0,  0,  0,  0],
        [-1,  0,  0, -1,  0,  0]
    ], [
        [ 0,  0, -1,  0,  0,  1],
        [ 0, -1,  0,  0, -1,  0],
        [ 1,  0,  0,  1,  0,  0],
        [ 0,  0,  0,  0,  0,  0]
    ]
])
print(T.shape)

(3, 4, 6)


In [9]:
cov_joined = linalg.block_diag(cov_a, cov_a_prime)
print(cov_joined)

V_0 = np.zeros([3, 3, T.shape[1], T.shape[1]])
for k, l in product(range(3), repeat=2):
    V_0[k, l] = T[k] @ cov_joined @ T[l].T
print(V_0.shape)

[[ 1.         -0.20386903 -0.86927576  0.          0.          0.        ]
 [-0.20386903  1.         -0.7617719   0.          0.          0.        ]
 [-0.86927576 -0.7617719   9.          0.          0.          0.        ]
 [ 0.          0.          0.          4.          1.68275463  1.62228044]
 [ 0.          0.          0.          1.68275463  1.          0.89673617]
 [ 0.          0.          0.          1.62228044  0.89673617  1.        ]]
(3, 3, 4, 4)


## 5.4 FNS法による最適化

In [10]:
def calc_M(W, Xi):
    dim = Xi.shape[1]
    M = np.zeros([dim, dim])
    for k, l in product(range(3), repeat=2):
        M += W[k, l] * Xi[k] @ Xi[l].T
    return M

In [11]:
def calc_L(W, q, Xi, V_0):
    _, dim, points_num = Xi.shape
    V = np.zeros([3, points_num])
    for k, l in product(range(3), repeat=2):
        V[k] += W[k, l] * Xi[l].T @ q
    L = np.zeros([dim, dim])
    for k, l in product(range(3), repeat=2):
        L += V[k].T @ V[l] * V_0[k, l]
    return L

In [12]:
def FNS_method(Xi, V_0):
    # step 1
    q0 = np.zeros(4)
    W = np.eye(3)
    iters = 1

    while True:
        # step 2
        X = calc_M(W, Xi) - calc_L(W, q0, Xi, V_0)
        # step 3
        w, eigenvecs = linalg.eigh(X)
        q = eigenvecs[:, np.argmin(w)]
        # step 4
        if np.allclose(q, q0) or np.allclose(q, -q0):
            return q, iters
        W_inv = np.zeros_like(W)
        for k, l in product(range(3), repeat=2):
            W_inv[k, l] = np.inner(q, V_0[k, l] @ q)
        W = linalg.inv(W_inv)
        q0 = q
        iters += 1

In [13]:
q, iters = FNS_method(Xi, V_0)
R2 = Rotation.from_quat(q[[1, 2, 3, 0]]).as_dcm()
print('iterations:', iters)
print('error:', util.eval_R_error(R2, ideal_R))

iterations: 4
error: 0.0007630886756842184


In [ ]:
util.plot_3d_multi(R2 @ A, A_prime)

## 5.5 同次拘束条件による解法

In [14]:
zeros = np.zeros([3, points_num])
Xi = np.stack([
    np.concatenate([A, zeros, zeros, -A_prime[[0]]]),
    np.concatenate([zeros, A, zeros, -A_prime[[1]]]),
    np.concatenate([zeros, zeros, A, -A_prime[[2]]])
])
del zeros
print(Xi.shape)

(3, 10, 40251)


In [15]:
T = np.zeros([3 ,10, 6])
for i in range(3):
    T[i, i * 3, 0] = T[i, i * 3 + 1, 1] = T[i, i * 3 + 2, 2] = 1
    T[i, 9, 3 +  i] = -1
print(T.shape)
print(T)

(3, 10, 6)
[[[ 1.  0.  0.  0.  0.  0.]
  [ 0.  1.  0.  0.  0.  0.]
  [ 0.  0.  1.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0. -1.  0.  0.]]

 [[ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 1.  0.  0.  0.  0.  0.]
  [ 0.  1.  0.  0.  0.  0.]
  [ 0.  0.  1.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0. -1.  0.]]

 [[ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.]
  [ 1.  0.  0.  0.  0.  0.]
  [ 0.  1.  0.  0.  0.  0.]
  [ 0.  0.  1.  0.  0.  0.]
  [ 0.  0.  0.  0.  0. -1.]]]


In [16]:
V_0 = np.zeros([3, 3, T.shape[1], T.shape[1]])
for k, l in product(range(3), repeat=2):
    V_0[k, l] = T[k] @ cov_joined @ T[l].T
print(V_0.shape)

(3, 3, 10, 10)


In [17]:
def projection_matrix(u):
    orthogonal_basis = np.array([
        [u[1], u[0], 0, u[4], u[3], 0, u[7], u[6], 0, 0],
        [0, u[2], u[1], 0, u[5], u[4], 0, u[8], u[7], 0],
        [u[2], 0, u[0], u[5], 0, u[3], u[8], 0, u[6], 0],
        [2*u[0], 0, 0, 2*u[3], 0, 0, 2*u[6], 0, 0, -2*u[9]],
        [0, 2*u[1], 0, 0, 2*u[4], 0, 0, 2*u[7], 0, -2*u[9]],
        [0, 0, 2*u[2], 0, 0, 2*u[5], 0, 0, 2*u[8], -2*u[9]],
    ]).T

    constraint_num = orthogonal_basis.shape[1]
    # Gram–Schmidt process
    Q, _ = linalg.qr(orthogonal_basis)
    P = np.eye(10)
    for i in range(6):
        P -= np.outer(Q[:, i], Q[:, i])
    return P, constraint_num

In [18]:
def EFNS_method(Xi, V_0):
    # step 1
    u = np.array([1., 0., 0.,
                  0., 1., 0.,
                  0., 0., 1., 1.])
    u /= linalg.norm(u)
    W = np.eye(3)
    iters = 1

    while True:
        # step 2
        M = calc_M(W, Xi)
        L = calc_L(W, u, Xi, V_0)
        # step 3, 4
        P, constraint_num = projection_matrix(u)
        # step 5
        X = P @ (M - L) @ P
        # step 6
        w, vecs = linalg.eigh(X)
        vecs = vecs[:, np.argsort(w)[:constraint_num + 1]]
        # step 7
        u_hat = np.zeros_like(u)
        for i in range(constraint_num + 1):
            u_hat += np.inner(u, vecs[:, i]) * vecs[:, i]
        # step 8
        u_prime = P @ u_hat
        u_prime /= linalg.norm(u_prime)
        if np.allclose(u_prime, u) or np.allclose(u_prime, -u):
            return u_prime, iters
        u += u_prime
        u /= linalg.norm(u)
        W_inv = np.zeros_like(W)
        for k, l in product(range(3), repeat=2):
            W_inv[k, l] = np.inner(u, V_0[k, l] @ u)
        W = linalg.inv(W_inv)
        iters += 1

In [19]:
u, iters = EFNS_method(Xi, V_0)
R3 = u[:-1].reshape(3, 3) / u[-1]
print('iterations:', iters)
print('error:', util.eval_R_error(R3, ideal_R))

iterations: 25
error: 0.0007630894932364349


In [ ]:
util.plot_3d_multi(R3 @ A, A_prime)

## 6.6 最尤推定による回転の最適化
（章は違うがやっていることは同じなので混ぜた）

In [20]:
def calc_W(cov_a, cov_a_prime, R):
    return linalg.inv(R @ cov_a @ R.T + cov_a_prime)

In [21]:
def calc_g(A, A_prime, R, W, cov_a):
    WE = W @ (A_prime - R @ A)
    g = (-np.cross(R @ A, WE, axis=0) + np.cross(WE, R @ cov_a @ R.T @ WE, axis=0)).sum(axis=1)
    return g

In [22]:
def calc_H(A, R, W):
    tmp = np.stack([
        np.cross(R @ A, W[:, 0], axis=0),
        np.cross(R @ A, W[:, 1], axis=0),
        np.cross(R @ A, W[:, 2], axis=0),
    ], axis=1)
    return np.cross(tmp, R @ A, axisa=1, axisb=0, axisc=1).sum(axis=2)

In [23]:
def calc_J(A, A_prime, cov_a, cov_a_prime, R):
    W = calc_W(cov_a, cov_a_prime, R)
    E = A_prime - R @ A
    return (E * (W @ E)).sum()

In [24]:
def lie_optimize(A, A_prime, cov_a, cov_a_prime):
    # step 1
    R = init_R = util.estimate_R_using_SVD(A, A_prime)
    J = init_J = calc_J(A, A_prime, cov_a, cov_a_prime, R)
    c = 0.0001

    while True:
        W = calc_W(cov_a, cov_a_prime, R)
        # step 2
        g = calc_g(A, A_prime, R, W, cov_a)
        H = calc_H(A, R, W)
        while True:
            # step 3
            omega = linalg.solve(H + c * np.eye(3), -g)
            # step 4
            new_R = util.exponential_map(omega) @ R
            # step 5
            new_J = calc_J(A, A_prime, cov_a, cov_a_prime, new_R)
            if new_J <= J:
                break
            c *= 10
        # step 6
        if linalg.norm(omega) < 1e-10:
            return new_R, new_J, init_R, init_J
        R = new_R
        J = new_J
        c /= 10

In [25]:
R4, J, init_R, init_J = lie_optimize(A, A_prime, cov_a, cov_a_prime)
print('initial error:', util.eval_R_error(R1, ideal_R))
print('final error:', util.eval_R_error(R4, ideal_R))
print('J:', init_J, '->', J)

initial error: 0.0011221782784201111
final error: 0.0007631184669944144
J: 1.0930399418220798 -> 1.0930326702424036


In [ ]:
util.plot_3d_multi(R4 @ A, A_prime)